<a href="https://colab.research.google.com/github/GyuhoonK/munhwa_bigdata/blob/master/shinhan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive')

#Setting matplot possible type Korean
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

import pickle
import os
import glob

import numpy as np
import pandas as pd
import re
from collections import defaultdict
from tqdm import tqdm_notebook
import datetime

import matplotlib.pyplot as plt

from math import ceil

DATA_PATH = '/content/gdrive/My Drive/Colab Notebooks/munhwa_bigdata'
SHINHAN_PATH = '/content/gdrive/My Drive/Colab Notebooks/munhwa_bigdata/shinhan'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Selecting previously unselected package fonts-nanum.
(Reading database ... 144487 files and directori

In [ ]:
native_2019 = pd.read_csv(os.path.join(SHINHAN_PATH, 'KCTI_NATIVE_2019.txt'),
                          sep = '|', encoding = 'CP949')

In [ ]:
tourist_2019 = native_2019[(native_2019.v1 != native_2019.v2) & (native_2019.gb3 == '여행')]

In [ ]:
tour_sum = tourist_2019.groupby(['v2','gb2','sex_ccd','cln_age_r','ta_ym','daw_ccd_r','apv_ts_dl_tm_r']).sum().reset_index()
city = tour_sum.v2.unique()

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def show_city_vlm_ratio(tour_sum, variable):
  vis = tour_sum.groupby(['v2','gb2']).sum().reset_index()
  vis['vlm_ratio'] = 0
  for city in vis.v2.unique():
    city_total_vlm = sum(vis[vis.v2 == city][variable])
    vis.loc[vis[vis.v2 == city].index, 'ratio'] = vis.loc[vis[vis.v2 == city].index, variable]/city_total_vlm
  fig = px.bar(vis, x = 'v2', y = 'ratio' , color = 'gb2',
               width = 10000,
              barmode = 'group')
  fig.update_layout(yaxis_title = 'vlm(%)',
                    xaxis_title = '도시',
                    title = '도시별 {} 비율 비교'.format(variable),
                    height = 800, width = 1200)
  fig.show()
show_city_vlm_ratio(tour_sum, 'vlm')

In [ ]:
 def show_city_by_age_vlm(tour_sum, variable):
  fig = make_subplots(rows = 5, cols = 1,
                      subplot_titles = list(map(str, tour_sum.cln_age_r.unique())),
                      )
  color_ = px.colors.qualitative.Dark24
  vis = tour_sum.groupby(['v2','gb2','cln_age_r'])
  for row, age in enumerate(tour_sum.cln_age_r.unique(), 1):
    if row == 1 : showlegend = True
    else : showlegend = False
    for city in vis.v2.unique():
      city_total_vlm = sum(vis[vis.v2 == city]['vlm'])
      vis.loc[vis[vis.v2 == city].index, 'vlm_ratio'] = vis.loc[vis[vis.v2 == city].index, 'vlm']/city_total_vlm
    for i, gb2 in enumerate(vis.gb2.unique()):
      vis_gb2 = vis[vis.gb2 == gb2]
      fig.add_trace(go.Bar(name = gb2,
                          x = vis_gb2.v2,
                          y = vis_gb2.vlm_ratio,
                          showlegend = showlegend,
                          marker_color = color_[i],
                          ),
                    row = row, col = 1)

  fig.update_layout(height = 800, width = 1600)
  fig.show() 
show_city_by_age_vlm(tour_sum)  

In [ ]:
def show_city_with_age_vlm_ratio(vis, age):
  for row, age in enumerate(tourist_2019.cln_age_r.unique(), 1):
    if row == 1 : showlegend = True
    else : showlegend = False
    for city in vis.v2.unique():
      city_total_vlm = sum(vis[vis.v2 == city]['vlm'])
      vis.loc[vis[vis.v2 == city].index, 'vlm_ratio'] = vis.loc[vis[vis.v2 == city].index, 'vlm']/city_total_vlm
    for i, gb2 in enumerate(vis.gb2.unique()):
      vis_gb2 = vis[vis.gb2 == gb2]
      fig.add_trace(go.Bar(name = gb2,
                          x = vis_gb2.v2,
                          y = vis_gb2.vlm_ratio,
                          showlegend = showlegend,
                          marker_color = color_[i],
                          ),
                    row = row, col = 1)

  fig.update_layout(height = 800, width = 1600)
  fig.show() 
show_city_with_age_vlm_ratio(vis, 20)

In [ ]:
gb2_target = ['외식', '종합쇼핑', '목욕', '레저스포츠', '미용', '패션쇼핑', '공연관람', '유흥', '사진촬영',
       '교육훈련', '인터넷게임', '헬스', '애완동물돌보기', '독서', '골프', '스포츠용품구매', '종교활동',
       '악기연주', '미술공예참여', '음악감상', '자전거', '스키', '운동경기관람']

drop_target = ['패션쇼핑','외식','종합쇼핑','인터넷게임','종교활동','음악감상','악기연주','교육훈련','헬스']

for i in drop_target:
  gb2_target.remove(i)

In [ ]:
tourist_2019_notour = native_2019[(native_2019.v1 != native_2019.v2) & (native_2019.gb2.isin(gb2_target))]
tour_sum = tourist_2019_notour.groupby(['v2','gb2','sex_ccd','cln_age_r','ta_ym','daw_ccd_r','apv_ts_dl_tm_r']).sum().reset_index()
city = tour_sum.v2.unique()
vis_notour = tour_sum.groupby(['v2','gb2']).sum().reset_index()

In [ ]:
show_city_vlm_ratio(tour_sum, 'vlm')

In [ ]:
show_city_by_age_vlm(vis_notour)